In [1]:
# import libraries
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
import kerastuner
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Flatten

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range= 0.2,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    rescale= 1/255.0
)
test_datagen = ImageDataGenerator(
    rescale= 1/255.0
)

In [ ]:
train_images = train_datagen.flow_from_directory("Dataset", target_size=(256,256), color_mode= "rgb")
test_images = test_datagen.flow_from_directory("validation_dataset", target_size=(256,256), color_mode= "rgb")

In [ ]:
# using keras tuner

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(Conv2D(32, (3,3),activation = "relu", input_shape = (256,256,3)))
    model.add(MaxPool2D((2,2)))       
    for i in range(hp.Int('CNN_layers', 2, 20)):
        model.add(Conv2D(hp.Choice('conv2D',[32,64,128]), (3,3),activation = "relu"))
        model.add(MaxPool2D((2,2)))
    model.add(Flatten()) 
    for i in range(hp.Int("Ann_layers", 2,20)):
        model.add(Dense(hp.Int('Dense', min_value = 32,max_value = 256,step = 32), activation = "relu"))
    model.add(Dense(2, activation = "sigmoid"))
    model.compile(optimizer= "adam", loss= "binary_crossentropy", metrics = ["accuracy"])
#     model.fit(train_images, epochs= 5, validation_data= test_images)   
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='CNN',
    project_name='First')

In [ ]:
tuner.search(train_images, train_images_labels
             epochs=5,
             validation_data=(test_images, test_images_labels))

In [ ]:
best_hyperparameter=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
model = tuner.hypermodel.build(best_hyperparameter)

In [ ]:
model.fit(X_train, y_train, epochs = 100, validation_split= 0.3)